In [1]:
import numpy as np
import json
from config import config
import db_module as dm
import gsheets as gs
from closed_date import add_cdate_var
import gspread
from oauth2client.client import SignedJwtAssertionCredentials
import pandas as pd
pd.options.mode.chained_assignment = None
import zipfile
import re
from datetime import date
import psycopg2

### Import relevant tables from DB

In [2]:
# db_mudule.py
#
#from config import config
#import pandas as pd
#import psycopg2
#
#class DBConn(object):
#    '''This is a class for querying the database and returning a pandas dataframe.'''
#    def __init__(self):
#        params = config()
#        conn = psycopg2.connect(**params)
#        self.cur = conn.cursor()
#    def ex_query(self, query):
#        query = query
#        cur = self.cur
#        cur.execute(query)
#        colnames = [desc[0] for desc in cur.description]
#        rows = cur.fetchall()
#        cur.close()
#        return pd.DataFrame(rows, columns=colnames)

cif = dm.DBConn()
query = str(
    "SELECT * FROM dataentry_cifnepal as CIF inner join \
    dataentry_personboxnepal as PB on CIF.id = PB.cif_id;")
db_cif = cif.ex_query(query)

vics = dm.DBConn()
query = str(
    "SELECT * FROM public.dataentry_person as p inner join \
    dataentry_cifnepal as CIF on p.id = CIF.main_pv_id;")
db_vics = vics.ex_query(query)

sus = dm.DBConn()
query = str(
    "SELECT * FROM public.dataentry_personboxnepal as pb inner join \
    public.dataentry_person as p on pb.person_id = p.id;")
db_sus = sus.ex_query(query)

add = dm.DBConn()
query = str(
    "SELECT * FROM public.dataentry_address1 as ad1 inner join \
    public.dataentry_address2 as ad2 on ad1.id = ad2.address1_id;")
db_add = add.ex_query(query)

### Subset data from CIFs and create suspect and victim IDs

In [3]:
cif_dates = db_cif[['cif_number', 'interview_date']]

cif_ids = db_cif[['cif_number', 'person_id', 'pb_number']]

add = db_add.iloc[:,[1,6,7]]
acols = ['address_1',
         'address2_id',
         'address_2']
add.columns = acols

db_vics.infer_objects
db_vics['address1_id'] = db_vics['address1_id'].fillna(0).astype(int)
db_vics['address2_id'] = db_vics['address2_id'].fillna(0).astype(int)
db_vics = pd.merge(db_vics, add, how='left',on='address2_id')
db_vics['Address'] = db_vics['address_2'].map(str) + ", " + db_vics['address_1']
cif_vics = db_vics[['cif_number',
                    'full_name',
                    'phone_contact',
                    'Address']]

db_sus.infer_objects
db_sus['address1_id'] = db_sus['address1_id'].fillna(0).astype(int)
db_sus['address2_id'] = db_sus['address2_id'].fillna(0).astype(int)
db_sus = pd.merge(db_sus, add, how='left', on='address2_id')
db_sus['Address'] = db_sus['address_2'].map(str) + ", " + db_sus['address_1']
db_sus = db_sus[['person_id', 'full_name', 'phone_contact', 'Address']]
cif_sus = pd.merge(db_sus, cif_ids, how='outer', on='person_id', sort=True,
         suffixes=('x', 'y'), copy=True)
cif_sus.loc[:,'pb_number']=cif_sus['pb_number'].fillna(0).astype(int)
cif_sus.loc[:,'Suspect_ID'] = cif_sus.loc[:,'cif_number'].str.replace('.','')
cif_sus.loc[:,'Suspect_ID'] = cif_sus.loc[:,'Suspect_ID'].str[:-1] + ".PB" + cif_sus['pb_number'].map(str)
cif_sus = cif_sus.drop_duplicates(subset='Suspect_ID')

cif_vics.loc[:, 'Victim_ID'] = cif_vics['cif_number']
replacements = {
   'Victim_ID': {
      r'(\.1|A$)': '.V1',r'B$': '.V2',r'C$': '.V3', r'D$': '.V4',r'E$': '.V5',
      r'F$': '.V6',r'G$': '.V7',r'H$': '.V8',r'I$': '.V9',r'J$': '.V10'}
}
cif_vics.replace(replacements, regex=True, inplace=True)
cif_vics.sort_values('full_name',inplace=True)
cif_vics = cif_vics.drop_duplicates(subset='Victim_ID')
non_blanks = cif_vics['full_name'] != ""
cif_vics = cif_vics[non_blanks]

cif_sus.loc[:, 'cif_number'] = cif_sus['cif_number'].str.replace('.','')
cif_vics['cif_number'] = cif_vics['cif_number'].str.replace('.','')
cif_vics['cif_number'] = cif_vics['cif_number'].str[:-1]
cif_sus['cif_number'] = cif_sus['cif_number'].str[:-1]

### Get current Case Dispatcher data from Google Sheets

In [4]:
# gsheets.py module (part 1):
#
# import gspread
# import json
# from oauth2client.client import SignedJwtAssertionCredentials
# import pandas as pd
# import re
#
# def get_gsheets(workbook_name):
#    '''Return a list of Google worksheets from the name of a Google Sheet.'''
#    json_key = json.load(open('creds.json'))
#    scope = ['https://spreadsheets.google.com/feeds',
#             'https://www.googleapis.com/auth/drive']
#    credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'].encode(), scope)
#    file = gspread.authorize(credentials)  #remember to share new sheets with client email
#    workbook = file.open(workbook_name)
#    gsheets = workbook.worksheets()
#    return gsheets

In [5]:
cdws = gs.get_gsheets("Case Dispatcher 2.0")

In [7]:
# gsheets.py module (part 2):
#
#class GSheet:
#    '''This is a class for Google Worksheets.'''
#    def __init__(self, wrksht):
#        self.wrksht = wrksht
#        self.name = re.findall(r"'(.*?)'", str(wrksht))[0]
#        df = pd.DataFrame(self.wrksht.get_all_values())
#        df.columns = df.iloc[0]
#        df.drop(0, inplace=True)
#        self.df = df

all_sheets = []
for i in range(len(cdws)):
        sheet = gs.GSheet(cdws[i])
        all_sheets.append(sheet)

dfs = {sheet.name: sheet.df for sheet in all_sheets}
locals().update(dfs)


# Tried to add the above functionality as a method in Gsheet class but it hasn't worked yet:
#    def get_df(self):
#        globals['{}'.format(self.name)] = self.df

today = date.today().strftime("%m-%d-%Y")
zipname = today + "_Backup_Sheets.zip"

with zipfile.ZipFile(zipname, 'w') as csv_zip:
    for k,v in dfs.items():
        fname= k + "_" + today + ".csv"
        csv_zip.writestr(fname, pd.DataFrame(v).to_csv())

In [8]:
# Add new cases from CIFs to CD data

new_suspects = cif_sus.iloc[:,[1, 2, 3, 4, 6]]
new_suspects.rename(columns = {
    'full_name':'Name',
    'phone_contact': 'Phone_Number(s)',
    'cif_number': 'Case_ID'},
           inplace=True)
new_police = new_suspects
new_suspects = new_suspects.reindex(columns=new_suspects.columns.tolist() + list(Suspects.columns))
new_suspects = new_suspects.iloc[:,5:len(new_suspects.columns)]
suspects = pd.concat([Suspects,new_suspects])
suspects = suspects.drop_duplicates(subset='Suspect_ID')

new_victims = cif_vics
vcols = ['Case_ID','Name',
         'Phone_Number(s)',
         'Address',
         'Victim_ID']
new_victims.columns = vcols
new_victims = new_victims.reindex( columns = new_victims.columns.tolist() + list(Victims.columns))
new_victims = new_victims.iloc[:,5:len(new_victims.columns)]
victims = pd.concat([Victims,new_victims])
victims = victims.drop_duplicates(subset='Victim_ID')

new_police.rename(columns = {'Name': 'Suspect_Name'})
new_police = new_police.reindex( columns = new_police.columns.tolist() + list(Police.columns))
new_police = new_police.iloc[:,5:len(new_police.columns)]
police = pd.concat([Police,new_police])
police = police.drop_duplicates(subset='Suspect_ID')

## Organize Arrest data from Case Dispatcher

arrests = pd.DataFrame(Arrests)
arrests.infer_objects()
arrests['Outcome (Arrest)'] = arrests['Outcome (Arrest)'].fillna(0).astype(int)
arrests = arrests.loc[arrests['Outcome (Arrest)'] == 1]

pbs = ['pb' + str(n) for n in range(1,8)]
for pb in pbs:
    arrests[pb + '_ID'] = arrests['IRF#'] + '.' + pb
for pb in pbs:
    arrests[pb + '_Case_ID'] = arrests['IRF#']

dpb={}
for pb in pbs:
    cnames = [col for col in arrests.columns if pb in col]
    dpb['df_{0}'.format(pb)]=pd.DataFrame(arrests[cnames])
newcn = ['Name',
         'Arrested',
         'Arrest_Date',
         'Suspect_ID',
         'Case_ID']
locals().update(dpb)

df_list = [df_pb1,
           df_pb2,
           df_pb3,
           df_pb4,
           df_pb5,
           df_pb6,
           df_pb7]

for i, df in enumerate(df_list, 1):
    df.columns = ['Name',
                  'Arrested',
                  'Arrest_Date',
                  'Suspect_ID',
                  'Case_ID']

df_pb_all = pd.concat(df_list)

arrests = df_pb_all[df_pb_all.Arrested.str.contains("Yes")]
arrests['Total_Arrests'] = arrests.groupby(['Case_ID'])['Case_ID'].transform('count')

### Move closed cases to closed sheets

In [9]:
prev_closed_sus = suspects[suspects.Suspect_ID.isin(arrests.Suspect_ID)]
prev_closed_pol = police[police.Suspect_ID.isin(arrests.Suspect_ID)]
prev_closed_sus['Case_Status'] = "Closed: Already in Legal Cases Sheet"
prev_closed_pol['Case_Status'] = "Closed: Already in Legal Cases Sheet"

closed_suspects = suspects[suspects['Date_Closed'].str.len() > 1]
closed_victims = victims[victims['Date_Closed'].str.len() > 1]
closed_police = police[police['Date_Closed'].str.len() > 1]

closed_cases = [closed_suspects,closed_victims,closed_police, prev_closed_sus, prev_closed_pol]

# closed_date.py
#
#def add_cdate_var(Sheets):
#    """Adds a variable with the current date to the end of each dataframe in a list."""
#    today = date.today()
#    for sheet in Sheets:
#        if len(sheet)>0:
#            sheet.loc[:,'Date_Closed'] = today.strftime("%m/%d/%Y")
#        else:
#            sheet['Date_Closed'] = ""

add_cdate_var(closed_cases)

closed_sus = pd.concat([Closed_Sus,closed_suspects, prev_closed_sus], sort=False)
closed_pol = pd.concat([Closed_Pol,closed_police, prev_closed_pol], sort=False)
closed_vic = pd.concat([Closed_Vic,closed_victims], sort=False)

# Next Step: Remove from Active Sheets
suspects = suspects[~suspects.Suspect_ID.isin(closed_sus.Suspect_ID)]
police = police[~police.Suspect_ID.isin(closed_pol.Suspect_ID)]
victims = victims[~victims.Victim_ID.isin(closed_vic.Victim_ID)]

closed_suspects = suspects[(suspects.Suspect_ID.isin(closed_pol.Suspect_ID)) |
                            (~suspects.Case_ID.isin(victims.Case_ID))]
closed_police = police[(police.Suspect_ID.isin(closed_sus.Suspect_ID)) |
                            (~police.Case_ID.isin(victims.Case_ID))]
closed_victims = victims[(~victims.Case_ID.isin(police.Case_ID)) |
                           (~victims.Case_ID.isin(suspects.Case_ID))]

closed_cases = [closed_suspects,
                closed_victims,
                closed_police]
add_cdate_var(closed_cases)

closed_sus = pd.concat([closed_sus,closed_suspects], sort=False).drop_duplicates(subset='Suspect_ID')
closed_vic = pd.concat([closed_vic,closed_victims], sort=False).drop_duplicates(subset='Victim_ID')
closed_pol = pd.concat([closed_pol,closed_police], sort=False).drop_duplicates(subset='Suspect_ID')
closed_pol.drop(columns=['Victims_Willing_to_Testify'])

suspects = suspects[~suspects.Suspect_ID.isin(closed_sus.Suspect_ID)]
police = police[~police.Suspect_ID.isin(closed_pol.Suspect_ID)]
victims = victims[~victims.Victim_ID.isin(closed_vic.Victim_ID)]

### Calculate Case Priority

In [10]:
victims['willing_to_testify'] = victims.Name[victims.Case_Status.str.contains("Step Complete", na=False)]
vics_willing = victims[['Case_ID','willing_to_testify']]
vics_willing = vics_willing.dropna(axis=0, subset=['willing_to_testify'])
vics_willing['count'] = 1

def sum_and_join(x):
     return pd.Series(dict(count = x['count'].sum(), 
                        willing_to_testify = ', '.join(x.astype(str)['willing_to_testify'])))
if len(vics_willing) > 0:
    vics_Willing = vics_willing.groupby('Case_ID').apply(sum_and_join)
    
police = pd.merge(police, vics_willing, how='left',on='Case_ID')

police.victims_willing_to_testify = police.willing_to_testify
police.drop(columns=['willing_to_testify', 'count'], inplace=True)

suspects = pd.merge(suspects, vics_willing, how='left',on='Case_ID')
v_multiplier = pd.DataFrame(Parameters.iloc[:10,6:8])
v_multiplier.Victims_Willing_to_Testify = v_multiplier.Victims_Willing_to_Testify.astype(int)

C:\Users\RandA\Anaconda3\envs\py36_32\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [11]:
suspects['count'] = suspects['count'].fillna(0).astype(int)
suspects = pd.merge(suspects, v_multiplier,how='left',left_on='count',right_on='Victims_Willing_to_Testify')
suspects.drop(columns=['Victims_Willing_to_Testify',
                       'willing_to_testify',
                       'count'], inplace=True)
suspects['V_Multiplier'].fillna(0, inplace=True)
suspects['V_Multiplier'] = suspects['V_Multiplier'].astype('float')
suspects['Bio_Known'] = np.where(suspects['Bio_and_Location'].eq(''),0,1)
suspects = pd.merge(suspects, arrests[['Case_ID','Total_Arrests']], how='left',on='Case_ID')
suspects['Total_Arrests'] = suspects['Total_Arrests'].fillna(0).astype(int)
suspects.rename(columns={'Total_Arrests':'Others_Arrested'}, inplace=True)
police['Willing_to_Arrest'] = np.where(police.Case_Status.str.contains("Step Complete", na=False),1,0)
suspects = pd.merge(suspects,police[['Case_ID','Willing_to_Arrest']], how='left', on='Case_ID')

today = date.today()
today.strftime("%m/%d/%Y")
cif_dates['Case_ID'] = cif_dates['cif_number'].str[:-1].replace('.','')
cif_dates['Days_Old'] = (today - cif_dates.loc[:,'interview_date']) / np.timedelta64(1, 'D')
suspects = pd.merge(suspects,cif_dates[['Case_ID','Days_Old']], how='left', on='Case_ID')
suspects['Recency_Score'] = np.where(suspects['Days_Old']<100, 1 - suspects.Days_Old * .01, 0)
suspects = suspects.drop_duplicates(subset='Suspect_ID')

# Get 'Strength of Case' results of CD module
soc = pd.read_csv("soc.csv")
suspects = pd.merge(suspects,soc,how='left',left_on='Suspect_ID',right_on='suspect_id')
suspects['Strength_of_Case'] = suspects['SOC'].round(decimals = 3)
suspects['Strength_of_Case']

suspects['Em2'] = suspects['Eminence'].fillna(1)
suspects.loc[suspects['Eminence'].str.len() < 1, 'Em2'] = 1
suspects['Em2'] = suspects['Em2'].astype(int)

In [12]:
weights_Vs = pd.Series(
    Parameters.iloc[0:7,1]).replace('',0).append(
    pd.Series(
        Parameters.iloc[0:3,5])).astype(float)

weights_Keys = pd.Series(
    Parameters.iloc[0:7,0]).append(
    pd.Series(
        Parameters.iloc[0:3,4]))

weights = {k:v for k,v in zip(weights_Keys, weights_Vs)}

In [13]:
suspects['Solvability'] = (
    suspects['V_Multiplier'].apply(lambda x: x * weights['Victim Willing to Testify']) + \
    suspects['Bio_Known'].apply(lambda x: x * weights['Bio and Location of Suspect']) + \
    suspects['Others_Arrested'].apply(lambda x: x * weights['Other Suspect(s) Arrested']) + \
    suspects['Willing_to_Arrest'].apply(lambda x: x * weights['Police Willing to Arrest']) + \
    suspects['Recency_Score'].apply(lambda x: x * weights['Recency of Case'])
)/sum(weights.values())

suspects['Priority'] = (
    suspects['Solvability'].apply(lambda x: x * weights['Solvability']) + \
    suspects['Strength_of_Case'].apply(lambda x: x * weights['Strength of Case']) + \
    suspects['Em2'].apply(lambda x: x * 0.1 * weights['Eminence'])
    ).round(decimals = 3)
suspects['Priority'] = suspects['Priority'].fillna(0)

### Upload results to Google Sheets

In [14]:
suspects['Priority'].astype(float)
suspects.sort_values('Priority',ascending=False, inplace=True)
suspects = suspects.iloc[:,0:len(Suspects.columns)].fillna('')

police = pd.merge(police, suspects[['Suspect_ID','Priority']])
police['Priority'].astype(float)
police.sort_values('Priority',ascending=False, inplace=True)
police = police.iloc[:,0:len(Police.columns)].fillna('')

victims = pd.merge(victims, suspects[['Case_ID','Priority']])
victims.sort_values('Priority',ascending=False, inplace=True)
victims = victims.iloc[:,0:len(Victims.columns)].fillna('')

In [15]:
suspects.name = 'suspects.csv'
police.name = 'police.csv'
victims.name = 'victims.csv'
closed_sus.name = 'closed_sus.csv'
closed_pol.name = 'closed_pol.csv'
closed_vic.name = 'closed_vic.csv'

all_sheets = [suspects,
              police,
              victims,
              closed_sus,
              closed_pol,
              closed_vic]

for sheet in all_sheets:
   sheet.to_csv(sheet.name,index=False,header=None)

In [17]:
up_sheets = []

for sheet in all_sheets:
    up_sheet = open(sheet.name,'r').read()
    up_sheets.append(up_sheet)
    
file = gspread.authorize(credentials)

ss = file.open("Suspects")
ps = file.open("Police")
vs = file.open("Victims")
css = file.open("Closed_Sus")
cps = file.open("Closed_Pol")
cvs = file.open("Closed_Vic")

gs = [ss,
      ps,
      vs,
      css,
      cps,
      cvs]

sheet_dict = {k:v for k,v in zip(gs, up_sheets)}

def upload_sheets(dict):
    """Uploads csv files to Google Sheets from a dictionary where keys are Google Sheets and values are csvs."""
    file = gspread.authorize(credentials)
    last = len(list(dict))
    for i in range(0,last):
        file.import_csv(
            list(
                dict.keys())[i].id,
            list(
                dict.values())[i].encode('utf-8'))

upload_sheets(sheet_dict)